In [1]:
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenizer_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenizer_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

### Training

1. 定义Trainer的第一步是构造一个TrainingArguments。该类包含了Trainer需要在训练和验证时用到的超参数。对我们而言只需要提供一个目录来保存模型，对基础微调，其它的参数只需要使用默认值就能取得非常好的效果

In [2]:
from transformers import TrainingArguments
training_args = TrainingArguments("test-trainer")

第二部是定义模型

In [3]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


为什么会产生警告？是因为bert还没有进行分类训练。原来bert的head被丢弃，新的适合分类任务的head被创建，所以显示有些参数没有被使用，而有些参数被随机初始化

In [6]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
)


注意：当processing_class使用tokenizer或者feature extractor是，Trainer将会默认使用DataCollatorWithPadding。所以此时你可以省略掉data_collator参数

In [7]:
trainer.train()

Step,Training Loss
500,0.641000
1000,0.634000


TrainOutput(global_step=1377, training_loss=0.6256421656390421, metrics={'train_runtime': 83.5979, 'train_samples_per_second': 131.63, 'train_steps_per_second': 16.472, 'total_flos': 405114969714960.0, 'train_loss': 0.6256421656390421, 'epoch': 3.0})

模型只是简单的每500步输出一下loss，这是因为：
1. 我们没有在TrainingArgument中告诉模型评估的策略(eval_strategy)
2. 我们没有给Trainer提供compute_metrics()

### Evaluation

使用`compute_metrics()`来评估我们的结果: 该函数必须使用`EvalPrediction`对象作为输入 (which is a named tuple with a predictions field and a label_ids field)，返回一个字典关于各个性能指标

In [9]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(type(predictions))
print(predictions.predictions.shape, predictions.label_ids.shape)

<class 'transformers.trainer_utils.PredictionOutput'>
(408, 2) (408,)


`predict()`的输出也是一个`named tuple`并包含有三个`key：predictions, label_ids, metrics.`目前metrics只包含loss和一些time metircs。当我们把`compute_metrics()`函数传给trainer后，metrics key中将包含compute_metircs返回的metrics

In [ ]:
# 注：named tuple就是一个轻量化的类，可通过member name访问元素
print(predictions.predictions)

[[-0.77104473  0.84788877]
 [-0.4133197   0.56212974]
 [ 0.02941573 -0.01366459]
 [-0.77004534  0.83597183]
 [-0.48264134  0.645507  ]
 [-0.7702328   0.8379393 ]
 [-0.5171508   0.67375934]
 [-0.77093977  0.84917945]
 [ 0.23693171 -0.09419888]
 [-0.770042    0.8185993 ]
 [-0.7694475   0.83842117]
 [-0.7453439   0.79224926]
 [ 0.31876194 -0.11711528]
 [-0.7672142   0.8098019 ]
 [-0.77101034  0.85131496]
 [-0.7706053   0.8188227 ]
 [-0.77046907  0.82761586]
 [ 0.3174038  -0.11644766]
 [-0.77112585  0.85269654]
 [ 0.29068673 -0.10477657]
 [ 0.31343925 -0.11438866]
 [-0.3701159   0.4454913 ]
 [-0.37589273  0.48357028]
 [-0.77022946  0.8211459 ]
 [-0.76219314  0.80503434]
 [-0.43866664  0.59915096]
 [-0.40537298  0.5523287 ]
 [-0.7703379   0.8467582 ]
 [-0.7697165   0.8415518 ]
 [-0.7297157   0.78435075]
 [-0.59408987  0.7223229 ]
 [-0.7703737   0.8410872 ]
 [-0.76992244  0.84563863]
 [-0.16157657  0.15769488]
 [-0.7413219   0.7914898 ]
 [-0.6929036   0.7669279 ]
 [-0.754981    0.799676  ]
 

In [15]:
print(predictions.label_ids)

[1 0 0 1 0 1 0 1 1 1 1 0 0 1 1 1 1 0 1 0 0 1 0 1 1 1 0 1 1 1 0 1 1 1 1 0 0
 0 1 1 0 1 0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1
 1 1 0 1 1 1 0 1 1 0 1 0 1 0 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0 1 1
 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 1 0 0 1 0 1 0 0 1 0 0 1 1
 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 0 1 1 0 1 1 0 0 1 1 0
 1 0 1 0 1 1 0 1 1 0 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 0 1 1 0 1 1 1 1 0 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 0 1 1 1 1 1 0 1 1 1 0 1 0 1 0 1 1 1
 0 1 0 1 1 0 1 1 0 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0
 0 0 1 1 1 1 1 1 1 1 0 1 0 0 0 1 1 0 1 0 1 1 0 0 0 0 0 1 0 1 0 1 1 1 1 1 0
 1 1 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 0 1 0 1 0 0 1 1 1 1 0 1 1 0 1 1 0 1 0 0
 1 1 1 1 0 0 0 0 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0
 1]


由于model的输出都是logits，因此我们需要将它转换为预测

In [14]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)
print(preds.shape)

(408,)


现在我们可以让preds与真实的label对比。我们可以通过evaluate库来获取对应dataset的metric.

In [17]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.7205882352941176, 'f1': 0.8167202572347267}

我们将上述操作打包在一起

In [18]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

现在我们创建一个新的 TrainingAruguments, 评估策略设置为每轮epoch

In [ ]:
training_args = TrainingArguments(
    "test-trainer", 
    eval_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.413831,0.828431,0.879310
2,0.548100,0.364082,0.862745,0.900709
3,0.342900,0.555680,0.872549,0.911565


TrainOutput(global_step=1377, training_loss=0.37735152677426587, metrics={'train_runtime': 98.0623, 'train_samples_per_second': 112.214, 'train_steps_per_second': 14.042, 'total_flos': 405114969714960.0, 'train_loss': 0.37735152677426587, 'epoch': 3.0})